### Opdracht 
We gaan proberen om een voorspelling te doen over de uitslag van een (verkleinde) voetbalpool om te zien wie er waarschijnlijk gaat winnen. We gaan deze voorspelling doen aan de hand van de  Random Number Generator en het principe vanMonte Carlo Simulatie.

Maak een Monte Carlo Simulatie die meerdere keren (duizenden) de competitie speelt met gebruik van deze tabel van kansen. Hoe vaker je een competitie speelt, hoe nauwkeuriger je voorspelling wordt. Houd bij hoeveel punten elk team scoort in de competitie (3 punten voor winst, 1 punt voor gelijk, 0 voor verlies), en maak een overzicht (ranking) van de teams.

<img src = "tabelMC.PNG">

In [44]:
import random
import pandas as pd
import numpy as np

Zet bovenstaande tabel om in een dictionary

In [2]:
clubs = {
    "ajax":{"feyenoord":[65,17,18], "psv":[54,21,25], "fcutrecht":[74,14,12], "willemII":[78,13,9]},
    "feyenoord": {"ajax": [30,21,49], "psv":[37,24,39], "fcutrecht":[51,22,27], "willemII":[60,21,19]},
    "psv": {"ajax": [39,22,39], "feyenoord":[54,22,24], "fcutrecht":[62,20,18], "willemII": [62,22,16]},
    "fcutrecht": {"ajax":[25,14,61], "feyenoord": [37,23,40], "psv":[29,24,47], "willemII": [53,23,25]},
    "willemII": {"ajax": [17, 18, 65], "feyenoord":[20,26,54], "psv":[23,24,53], "fcutrecht":[37,25,38]}
}

Een functie maken waar we willekeurige getallen genereren op basis van de Linear congruential method.
De formule voor de Linear Congruential method luid als volgt: <br>
**X n+1 = (a * Xn + c) mod m**
waar
* a= multiplier
* c = increment
* Xn als n = 0 => seed (starting value)
* m = modulus


In [3]:
def random_number_generator(n):
    "Generating a random number using linear congruential method "
    seed = 20
    a = 3
    c = 3
    m = 100
    
    x=[seed] # list containing n random generated numbers 
    for i in range(1,n+1):
        if i == 1:
            x_next = (a * seed + c) % m
            x.append(x_next)
        else:
            x_next = (a * x[i-1] + c) % m
            x.append(x_next)
    return x


Definieer een spel tussen 2 teams. <br>
De scores luidt als volgt:
* 3 punten voor winst
* 1 punt voor gelijk spel
* 0 punten voor verlies

In [4]:
def match(team_lst, rn):
    
    #vb ayax - feyenoord [65, 17, 18]
    if  rn <= team_lst[2]: # rn< 18 , dan verlies thuisspelende partij
        return 0, 3
    elif team_lst[2] < rn <= (team_lst[2] + team_lst [1]): # rn zit tussen 18 en  35 (18+17), dan is er een gelijkspel
        return 1,1
    elif  rn >=(team_lst[1]+ team_lst[2]): # rn zit tussen 35 en 100 (35 + 65), dan win de thuisspelende partij
        return 3, 0
    

Definieer competities waarbij een team thuis speelt met de rest van de teams (uitploegen).

In [5]:
#lijst met de scores van elke team
def competitions(hteam, rng):
    scores_teams = {
    "ajax" : 0,
    "feyenoord" : 0,
    "psv" : 0,
    "fcutrecht" : 0,
    "willemII" : 0
}
    #counter = 0
    oponents = clubs[hteam] # pakt de dictionary met de andere teams en de lijst van de win-/gelijkspel-/verlieskans met de teams
    for ateam in oponents:
        rn = random.choice(rng) # random getal uit de lijst van de random gegenereerde getallen 
        ht_point, at_point = match(oponents.get(ateam), rn)# pak de lijst van kansen voor de away team van de ene home team
        scores_teams[hteam] += ht_point # voeg aantal punten toe voor thuisspelende partij
        scores_teams[ateam] += at_point # voeg aantal punten toe voor uitspelende partij
        
        #counter +=1
    return scores_teams



In [6]:
rng  = random_number_generator(1000)
print(rng[:20], sep="") # print de eerste 20 getallen 

[20, 63, 92, 79, 40, 23, 72, 19, 60, 83, 52, 59, 80, 43, 32, 99, 0, 3, 12, 39]


Er wordt een championship functie gemaakt, waar alle teams thuis speelt.

In [7]:
def championships(prob_pos_clubs):
    "Each teams plays in a competition as hometeam"  

    for i in prob_pos_clubs.keys():
        scores_comp = competitions(i, rng)
        rank_comp   = {k: v for k, v in sorted(scores_comp.items(), key=lambda item: item[1], reverse= True)}# dictionary sorteren 
        # voeg ranking toe in dictionary prob_pos_clubs
        pos = 1
        for i in rank_comp.keys():
            prob_pos_clubs[i][pos] += 1
            pos +=1
# Bron : Dictionary sorteren 

Creeer Monte carlo functie die de functie championships n keren runt en de resultaten opslaan in prob_pos_clubs

In [38]:
def MC(n):
    # aantal 
    prob_pos_clubs = {
        "ajax":{1:0, 2:0, 3:0, 4:0, 5:0},
        "feyenoord": {1:0, 2:0, 3:0, 4:0, 5:0},
        "psv":{1:0, 2:0, 3:0, 4:0, 5:0},
        "fcutrecht":{1:0, 2:0, 3:0, 4:0, 5:0},
        "willemII":{1:0, 2:0, 3:0, 4:0, 5:0}
    }
    
    counter = 0
    while counter  < n:
        championships(prob_pos_clubs)
        counter +=1
    return prob_pos_clubs

In [42]:
n = 2
res = MC(n)

res

{'ajax': {1: 4, 2: 3, 3: 1, 4: 1, 5: 1},
 'feyenoord': {1: 2, 2: 4, 3: 3, 4: 0, 5: 1},
 'psv': {1: 2, 2: 1, 3: 4, 4: 2, 5: 1},
 'fcutrecht': {1: 1, 2: 1, 3: 2, 4: 5, 5: 1},
 'willemII': {1: 1, 2: 1, 3: 0, 4: 2, 5: 6}}

 Zet dictionaries om in een dataframe 

In [49]:
df = pd.DataFrame(res)
def calculate_probability(x):
    return (x / (n*25))*100
df2 = df.apply(np.vectorize(calculate_probability))
print("Amount of competition that each team has had to get to that place.")
display(df)
print("Probability of each team getting to that place in a competition.")
display(df2)

Amount of competition that each team has had to get to that place.


,ajax,feyenoord,psv,fcutrecht,willemII
1,4,2,2,1,1
2,3,4,1,1,1
3,1,3,4,2,0
4,1,0,2,5,2
5,1,1,1,1,6


Probability of each team getting to that place in a competition.


,ajax,feyenoord,psv,fcutrecht,willemII
1,8.0,4.0,4.0,2.0,2.0
2,6.0,8.0,2.0,2.0,2.0
3,2.0,6.0,8.0,4.0,0.0
4,2.0,0.0,4.0,10.0,4.0
5,2.0,2.0,2.0,2.0,12.0


Each champiosnhips has 5 competitions with 5 places(1st place, 2nd place etc), thus each championships has 25 places.